In [1]:
# Baixa o executável do AutoDock Vina versão 1.2.7
!wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.7/vina_1.2.7_linux_x86_64

# Concede permissão de execução ao arquivo baixado
!chmod +x vina_1.2.7_linux_x86_64

# Move o executável para um diretório acessível globalmente no ambiente
!mv vina_1.2.7_linux_x86_64 /usr/local/bin/vina

# Importa bibliotecas necessárias e testa a instalação exibindo o menu de ajuda
import os
import subprocess
from google.colab import files
!vina --help
import pandas as pd


--2025-08-20 08:27:25--  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.7/vina_1.2.7_linux_x86_64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/258054635/2c1a7c52-9d1a-4b51-acfa-60c2e805c70e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-20T09%3A11%3A05Z&rscd=attachment%3B+filename%3Dvina_1.2.7_linux_x86_64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-20T08%3A10%3A34Z&ske=2025-08-20T09%3A11%3A05Z&sks=b&skv=2018-11-09&sig=BhkXtsl%2FMUu8n2WMePH3XCNUgk%2BmnvmccAYrmXDbUo8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NTY3ODc0NiwibmJmIjoxNzU1Njc4NDQ2LCJwYXRoIjoicmVsZWF

In [2]:
import os
import subprocess
import pandas as pd

# Caminho para o arquivo do receptor preparado (formato PDBQT)
receptor_file = "1eqp_preparada.pdbqt"

# Lista de arquivos de ligantes preparados (formato PDBQT)
ligant_files = [
    "ZINC000299869729.pdbqt",
    "ZINC000009470583.pdbqt",
    "ZINC000034020906.pdbqt",
    "ZINC000034020907.pdbqt",
    "ZINC000299869730.pdbqt",
    "ZINC000009470585.pdbqt",
    "ZINC000196183247.pdbqt",
    "ZINC000196183259.pdbqt",
    "ZINC000038289836.pdbqt",
    "ZINC000038289837.pdbqt",
    "ZINC000141515017.pdbqt",
    "ZINC000103237407.pdbqt",
    "ZINC000103237409.pdbqt",
    "ZINC000141515219.pdbqt",
    "ZINC000504084519.pdbqt",
    "ZINC000040844993.pdbqt",
    "ZINC000040844994.pdbqt",
    "ZINC000504084521.pdbqt",
    "Fluconazole.pdbqt"
]

# Coordenadas do centro da caixa de busca
center = [34.0842, 31.9148, 57.8778]

# Tamanho da caixa de busca
size = [58.4224413061, 53.8679805946, 60.8922042465]

# Parâmetros de busca
exhaustiveness = 100
num_modes = 10

In [ ]:
# Criar diretório de saída
output_dir = "/content/docking_results"
os.makedirs(output_dir, exist_ok=True)

# Lista para armazenar os resultados
all_results = []

# Loop sobre cada ligante
for i, file in enumerate(ligant_files, 1):
    file_name = os.path.splitext(os.path.basename(file))[0]
    output_file = os.path.join(output_dir, f"{file_name}_output.pdbqt")
    log_file = os.path.join(output_dir, f"{file_name}_log.txt")

    print(f"\n[{i}/{len(ligant_files)}] Rodando docking para: {file_name}")

    cmd = [
        "vina",
        "--receptor", receptor_file,
        "--ligand", file,
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(size[0]),
        "--size_y", str(size[1]),
        "--size_z", str(size[2]),
        "--out", output_file,
        "--exhaustiveness", str(exhaustiveness),
        "--num_modes", str(num_modes)
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    # Salva o log
    with open(log_file, "w") as f:
        f.write(result.stdout)
        if result.stderr:
            f.write("\n\n[STDERR]\n" + result.stderr)

    # Verifica e processa saída
    if result.returncode == 0:
        print(f"✓ Docking concluído: {file_name}")

        for line in result.stdout.splitlines():
            if line.strip().startswith(tuple(str(i) for i in range(1, num_modes + 1))):
                parts = line.split()
                if len(parts) >= 2:
                    all_results.append({
                        "Ligand": file_name,
                        "Mode": parts[0],
                        "Affinity (kcal/mol)": parts[1],
                        "RMSD LB": parts[2] if len(parts) > 2 else "",
                        "RMSD UB": parts[3] if len(parts) > 3 else ""
                    })
    else:
        print(f"✗ ERRO no docking de {file_name}")
        all_results.append({
            "Ligand": file_name,
            "Mode": "ERROR",
            "Affinity (kcal/mol)": None,
            "RMSD LB": None,
            "RMSD UB": None
        })


[1/19] Rodando docking para: ZINC000299869729


In [ ]:
# Converte e salva resultados em CSV
df = pd.DataFrame(all_results)
csv_path = os.path.join(output_dir, "docking_summary.csv")
df.to_csv(csv_path, index=False)

print(f"\n✅ Docking finalizado para {len(ligant_files)} ligantes.")
print(f"Resultados salvos em: {csv_path}")

In [ ]:
import shutil

# Caminho da pasta com os resultados de docking
input_folder = '/content/docking_results'

# Nome do arquivo ZIP de saída
output_zip = '/content/docking_results.zip'

# Cria o ZIP contendo tudo da pasta
shutil.make_archive(base_name=output_zip.replace('.zip', ''), format='zip', root_dir=input_folder)

# Faz o download do ZIP
from google.colab import files
files.download(output_zip)